In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from hyperparameter_tuning import random_search
from feature_builder import process_dataset
import nltk

In [2]:
x_train = pd.read_csv('train.csv')
y_train=x_train[['id','target']]

In [4]:
#Parametros a usar de manera aleatoria
params = {

        'learning_rate': [ 0.05, 0.1, 0.15, 0.2, 0.25],
        'subsample': [0.7, 0.8, 0.9,  1.0],
        'colsample_bytree': [0.7, 0.8, 0.9,  1.0],
        'max_depth': [3, 4, 5, 6],
        'n_estimators': [ 600, 800, 900, 1000, 1100, 1200, 1300]
    }

Utilizo unicamente los features de feature_builder

In [5]:
x_train_features = process_dataset(x_train)

In [6]:
x_train_feat, x_test_feat, y_train_feat, y_test_feat = train_test_split(x_train_features, y_train, test_size = .33, random_state = 17)

In [7]:
xgb = XGBClassifier(objective='binary:logistic',
                    silent=True)

In [8]:
random_search(x_train_feat,y_train_feat['target'],xgb,params)

Fitting 5 folds for each of 220 candidates, totalling 1100 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  8.8min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed: 41.3min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed: 96.4min
[Parallel(n_jobs=4)]: Done 504 tasks      | elapsed: 171.6min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 272.6min
[Parallel(n_jobs=4)]: Done 1100 out of 1100 | elapsed: 368.3min finished


[08:00:56] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.



 Time taken: 6 hours 8 minutes and 32.2 seconds.

 Best f1 score with 5-folds and 220 combinations of hyperparameters:
0.7665666582437686

 Best hyperparameters:
{'subsample': 0.9, 'n_estimators': 600, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 0.9}


In [9]:
xgbooster = XGBClassifier(max_depth=3, n_estimators=600, colsample_bytree=0.9,
                        subsample=0.9, nthread=4, learning_rate=0.05)


xgbooster.fit(x_train_feat, y_train_feat['target'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=600, n_jobs=4, nthread=4, num_parallel_tree=1,
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=0.9, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [10]:
predictions_features = xgbooster.predict(x_test_feat)
accuracy_score(y_test_feat['target'], predictions_features)

0.8209311579785118

In [11]:
f1_score(y_test_feat['target'], predictions_features)

0.7791952894995092